In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf

# Symbols for ETFs
etfs = ['TLT','GLD','XLP','XLK','XLV']  

# Download historical data
data = yf.download(etfs, start="2005-01-01", end="2020-12-31")['Adj Close']
data.ffill(inplace=True)  # Forward-fill to handle missing values
data.head(5)


[*********************100%%**********************]  5 of 5 completed


Ticker,GLD,TLT,XLK,XLP,XLV
Date,,,,,
2005-01-03,43.020000,48.526787,16.243137,14.010854,21.538509
2005-01-04,42.740002,48.018280,15.940468,13.962288,21.365276
2005-01-05,42.669998,48.275242,15.886136,13.889449,21.329189
2005-01-06,42.150002,48.308052,15.839581,13.956213,21.487982
2005-01-07,41.840000,48.417439,15.870617,14.022999,21.473557


In [2]:
# Generating the Boolean series
last_day_of_month = data.groupby(pd.Grouper(freq='M')).apply(lambda x: x.index.max())
monthly_last_trading = data.index.isin(last_day_of_month)

# Convert to Series
exits = pd.Series(monthly_last_trading, index=data.index)

print(exits)


/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_35878/2342592832.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  last_day_of_month = data.groupby(pd.Grouper(freq='M')).apply(lambda x: x.index.max())


Date
2005-01-03    False
2005-01-04    False
2005-01-05    False
2005-01-06    False
2005-01-07    False
              ...  
2020-12-23    False
2020-12-24    False
2020-12-28    False
2020-12-29    False
2020-12-30     True
Length: 4027, dtype: bool


In [3]:
# # Generate last day of each month for the given period
# rebalance_dates = pd.date_range(start=data.index.min(), end=data.index.max(), freq='BM')

# # Define entries on the rebalance dates
# entries = pd.DataFrame(False, index=data.index, columns=data.columns)

# for date in rebalance_dates:
#     if date in entries.index:
#         entries.loc[date] = True

# print(entries.head(5))

# # Define exits - in this strategy, there are technically no exits, just rebalances
# exits = pd.DataFrame(False, index=data.index, columns=data.columns)
# print(exits.head(40))

# Weights for each ETF in the portfolio
weights = np.repeat(1/len(etfs), len(etfs))

# Create a portfolio with rebalancing on specific dates
portfolio = vbt.Portfolio.from_signals(
    data,
    entries=entries,
    exits=exits,
    size=weights,
    size_type='percent',  # Use 'percent' to specify weights as percentage of current portfolio value
    freq='D',
    init_cash=10000
)

# Run the backtest
results = portfolio.total_return()

# Output the results
print(results)



NameError: name 'entries' is not defined

In [ ]:
# Calculate daily returns
daily_returns = portfolio.returns()

# Calculate annual returns
annual_returns = daily_returns.resample('A').apply(lambda x: (x + 1).prod() - 1)

print("Annual Returns:")
print(annual_returns)


Annual Returns:
Ticker           GLD       TLT       XLK       XLP       XLV
Date                                                        
2005-12-31  0.044339  0.009714  0.011277  0.003305  0.019672
2006-12-31  0.052753  0.001471  0.025399  0.029362  0.015191
2007-12-31  0.082942  0.021522  0.035322  0.028592  0.016265
2008-12-31  0.016153  0.076571 -0.105775 -0.037095 -0.055838
2009-12-31  0.081402 -0.061191  0.085555  0.031111  0.037978
2010-12-31  0.113726  0.021065  0.026477  0.033278  0.007392
2011-12-31  0.043140  0.084843  0.006591  0.037460  0.028479
2012-12-31  0.031259  0.007426  0.039321  0.031366  0.043614
2013-12-31 -0.138713 -0.041927  0.074833  0.082597  0.116924
2014-12-31 -0.008912  0.077339  0.059762  0.057580  0.089859
2015-12-31 -0.042909 -0.005990  0.020370  0.027547  0.028039
2016-12-31  0.030147  0.003875  0.057791  0.020763 -0.011780
2017-12-31  0.050415  0.030606  0.143389  0.055656  0.091382
2018-12-31 -0.008204 -0.005692 -0.008243 -0.037033  0.029409
2019-12-

/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_35299/979315640.py:5: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_returns = daily_returns.resample('A').apply(lambda x: (x + 1).prod() - 1)


In [ ]:
print(portfolio.stats())

Start                         2005-01-03 00:00:00
End                           2020-12-30 00:00:00
Period                         4027 days 00:00:00
Start Value                               10000.0
End Value                            17868.391212
Total Return [%]                        78.683912
Benchmark Return [%]                   384.922813
Max Gross Exposure [%]                  54.560755
Total Fees Paid                               0.0
Max Drawdown [%]                        15.631748
Max Drawdown Duration          1056 days 19:12:00
Total Trades                                  1.0
Total Closed Trades                           0.0
Total Open Trades                             1.0
Open Trade PnL                        7868.391212
Win Rate [%]                                  NaN
Best Trade [%]                                NaN
Worst Trade [%]                               NaN
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                          NaN


/var/folders/yh/nnw5blt935v54t02pj41_rjm0000gn/T/ipykernel_35299/1076709553.py:1: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x1103e93a0>. Pass column to select a single column/group.
  print(portfolio.stats())
